In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import svd
%matplotlib widget

# ---------- load data ----------------------------------------------------
df = pd.read_csv('../csv/MSFT_M1_202402291729_202503272108.csv', sep='\t')  # Use tab as the separator
df['Datetime'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])  # Use the correct column names
df.set_index('Datetime', inplace=True)
df.drop(columns=['<DATE>', '<TIME>'], inplace=True)  # Drop the original columns
# Rename columns to match what backtrader expects (optional, if needed)
df.rename(columns={'<OPEN>': 'Open', '<HIGH>': 'High', '<LOW>': 'Low', '<CLOSE>': 'Close', '<TICKVOL>': 'TickVol', '<VOL>': 'Volume'}, inplace=True)

# ---------- SSA helpers --------------------------------------------------
def ssa_decompose(series, window):
    K = len(series) - window + 1
    X = np.vstack([series[i:i+K] for i in range(window)])
    U, s, Vt = svd(X, full_matrices=False)
    return U, s, Vt, window

def reconstruct_trend(U, s, Vt, window, idx):
    if isinstance(idx, int):
        idx = [idx]
    Xn = (U[:, idx] * s[idx]) @ Vt[idx, :]
    M, K = Xn.shape
    N = M + K - 1
    recon = np.zeros(N)
    counts = np.zeros(N)
    for i in range(M):
        recon[i:i+K] += Xn[i]
        counts[i:i+K] += 1
    return recon / counts

# ---------- parameters ---------------------------------------------------
series_all = df['Close'].to_numpy()

window      = 60
k_trend     = list(range(5))
initial_len = 3000
step        = 1
n_updates   = 1000

last_vals  = []
last_times = []

# ---------- build "black line" ------------------------------------------
from tqdm.notebook import tqdm
for i in tqdm(range(n_updates)):
    end = initial_len + i * step
    if end > len(series_all):
        break
    seg = series_all[:end]
    U, s, Vt, win = ssa_decompose(seg, window)
    trend = reconstruct_trend(U, s, Vt, win, k_trend)
    last_times.append(end - 1)
    last_vals.append(trend[-1])
    if i % 10 == 0:
        print(f"i = {i}, end = {end}, trend[-1] = {trend[-1]:.2f}", flush=True)

last_vals  = np.array(last_vals)
last_times = np.array(last_times)

# ---------- trading signals ---------------------------------------------
price   = series_all[last_times]
ret     = np.diff(price) / price[:-1]          # simple returns
slope   = np.diff(last_vals)

pos      = np.zeros_like(ret)
cur_pos  = 1 if slope[0] >= 0 else -1
pos[0]   = cur_pos
flips    = 0


for i in tqdm(range(1, len(ret))):
    if slope[i] * slope[i-1] < 0:              # смена знака ∂Y
        cur_pos *= -1
        flips   += 1
    pos[i] = cur_pos

str_ret   = pos * ret
equity    = np.cumprod(1 + str_ret)
bh_equity = np.cumprod(1 + ret)

# ---------- plot ---------------------------------------------------------
plt.figure(figsize=(10, 5))
plt.plot(equity,     label="SSA-reversal strategy")
plt.plot(bh_equity,  label="Buy & hold")
plt.title("Equity curve: flip on black-line reversals")
plt.xlabel("Trade index")
plt.ylabel("Cumulative growth")
plt.grid(True); plt.legend(); plt.tight_layout()
plt.show()

print(f"Total return strategy: {equity[-1]-1:.2%}")
print(f"Total return buy&hold: {bh_equity[-1]-1:.2%}")
print(f"Number of flips (trades): {flips}")
